In [ ]:
import pandas as pd
import json
import os

# 경로 설정
BASE_DIR = os.path.dirname(os.getcwd())
CSV_PATH = os.path.join(BASE_DIR, "data", "HW_LDGS_PET_INFO_202412.csv")
JSON_PATH = os.path.join(BASE_DIR, "data", "pet_lodging_places_202412.json")

# 폴더가 없으면 생성
os.makedirs(os.path.dirname(JSON_PATH), exist_ok=True)

# CSV 읽기
df = pd.read_csv(CSV_PATH, encoding="utf-8")

# 문자열 정리
df['PET_INFO_CN'] = df['PET_INFO_CN'].fillna("").str.strip()

# 변환 결과 저장 리스트
documents = []

for _, row in df.iterrows():
    pet_info = row['PET_INFO_CN']

    # 조건: "불가" 포함 + "보조" 미포함 → 제외
    if '불가' in pet_info and '보조' not in pet_info:
        continue

    province = row['CTPRVN_NM']
    city = row['GUGUN_NM'] if pd.notnull(row['GUGUN_NM']) else ""
    content = (
        f"{province} {city}에 위치한 '{row['LDGS_NM']}'은(는) 반려동물 동반 가능한 숙박업소입니다. "
        f"주소는 '{row['LDGS_ADDR']}'이고, 반려동물 조건은 '{pet_info}'입니다."
    )
    metadata = {
        "facility_name": row['LDGS_NM'],
        "province": province,
        "city": city,
        "road_address": row['LDGS_ADDR'],
        "category": "숙박업소",
        "pet_info": pet_info
    }
    documents.append({
        "content": content,
        "metadata": metadata
    })

# JSON 저장
with open(JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(documents, f, ensure_ascii=False, indent=2)

print(f"✅ 숙박업소 JSON 변환 완료 (동물 완전 불가 제외): {JSON_PATH}")


✅ 숙박업소 JSON 변환 완료: c:\Users\treze\OneDrive\바탕 화면\3rd1st\data\pet_lodging_places_202412.json
